In [1]:
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [2]:
# !pip install torchvision
# !pip install --upgrade nvidia-pyindex
# !nvidia-smi
import torch

if torch.cuda.is_available():
    print("CUDA is available!")
    print(f"Number of GPUs: {torch.cuda.device_count()}")
    print(f"Current GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"PyTorch CUDA version: {torch.version.cuda}")
else:
    print("CUDA is NOT available. PyTorch is using CPU.")

# You can also explicitly check the device variable assignment:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device variable set to: '{device}'")

CUDA is available!
Number of GPUs: 1
Current GPU Name: Tesla T4
PyTorch CUDA version: 12.4
Device variable set to: 'cuda'


In [3]:
import cv2
import os
import math

# --- Configuration ---
VIDEO_PATH = "/content/race.mp4"          # Path to your input video
OUTPUT_DIR = "extracted_frames"  # Directory to save the extracted image frames
DESIRED_FPS = 5                  # Annotate approximately 5 frames per second

# --- Frame Extraction ---
# Create the output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Open the video file
cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened():
    print(f"Error: Could not open video file: {VIDEO_PATH}")
    exit()

# Get video properties
video_fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_interval = 1 # Default to processing every frame if desired_fps is higher than video_fps or 0
if DESIRED_FPS > 0 and DESIRED_FPS < video_fps:
     frame_interval = math.ceil(video_fps / DESIRED_FPS) # Calculate how many frames to skip

print(f"Video FPS: {video_fps:.2f}")
print(f"Desired Annotation FPS: {DESIRED_FPS}")
print(f"Processing every {frame_interval} frame(s)")

frame_count = 0
saved_frame_count = 0

while True:
    ret, frame = cap.read()

    # If frame reading was not successful, break the loop
    if not ret:
        break

    # Check if the current frame should be processed based on the interval
    if frame_count % frame_interval == 0:
        # Construct the output filename (e.g., frame_00000.jpg)
        output_filename = os.path.join(OUTPUT_DIR, f"frame_{frame_count:05d}.jpg")
        # Save the frame as an image file
        cv2.imwrite(output_filename, frame)
        saved_frame_count += 1
        print(f"\rSaved frame {saved_frame_count} (Video Frame {frame_count}/{total_frames})", end="")

    frame_count += 1

# Release the video capture object
cap.release()
print(f"\nFinished extracting {saved_frame_count} frames to '{OUTPUT_DIR}'")

Video FPS: 30.00
Desired Annotation FPS: 5
Processing every 6 frame(s)
Saved frame 90 (Video Frame 534/538)
Finished extracting 90 frames to 'extracted_frames'


In [ ]:
from ultralytics.data.annotator import auto_annotate
import torch # Make sure torch is available for device selection

# --- Configuration ---
IMAGE_DATA_PATH = "extracted_frames" # Path to the directory created in Step 1

DET_MODEL_PATH = "/content/yolo11s-pose.pt"
SAM_MODEL_PATH = "/content/sam2_b.pt"
OUTPUT_ANN_DIR = "output_annotations" # Directory where annotation labels will be saved

# Determine device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# --- Auto Annotation ---
print(f"Starting auto-annotation on frames in: {IMAGE_DATA_PATH}")
auto_annotate(
    data=IMAGE_DATA_PATH,
    det_model=DET_MODEL_PATH,
    sam_model=SAM_MODEL_PATH,
    device=device, # Specify the device (GPU is recommended for speed)
    output_dir=OUTPUT_ANN_DIR # Specify where the annotations should be saved
)
print(f"Annotation process finished. Labels saved in '{OUTPUT_ANN_DIR}'")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Using device: cuda
Starting auto-annotation on frames in: extracted_frames


100%|██████████| 154M/154M [00:02<00:00, 59.2MB/s]



image 1/90 /content/extracted_frames/frame_00000.jpg: 384x640 1 person, 55.8ms
image 2/90 /content/extracted_frames/frame_00006.jpg: 384x640 1 person, 11.7ms
image 3/90 /content/extracted_frames/frame_00012.jpg: 384x640 2 persons, 10.4ms
image 4/90 /content/extracted_frames/frame_00018.jpg: 384x640 2 persons, 11.0ms
image 5/90 /content/extracted_frames/frame_00024.jpg: 384x640 2 persons, 10.0ms
image 6/90 /content/extracted_frames/frame_00030.jpg: 384x640 2 persons, 13.6ms
image 7/90 /content/extracted_frames/frame_00036.jpg: 384x640 2 persons, 13.3ms
image 8/90 /content/extracted_frames/frame_00042.jpg: 384x640 2 persons, 14.9ms
image 9/90 /content/extracted_frames/frame_00048.jpg: 384x640 2 persons, 13.0ms
image 10/90 /content/extracted_frames/frame_00054.jpg: 384x640 2 persons, 17.9ms
image 11/90 /content/extracted_frames/frame_00060.jpg: 384x640 2 persons, 17.3ms
image 12/90 /content/extracted_frames/frame_00066.jpg: 384x640 2 persons, 20.7ms
image 13/90 /content/extracted_frames/

In [ ]:
import torch
from ultralytics import YOLO
import os
import glob
import random
import shutil
import yaml # To write and read yaml

# --- Configuration ---

# 1. Source Data Paths
SOURCE_IMAGE_DIR = "extracted_frames"
SOURCE_LABEL_DIR = "output_annotations" # Directory containing the .txt label files

# 2. Dataset Organization Paths
DATASET_BASE_DIR = "dataset"
TRAIN_IMG_DIR = os.path.join(DATASET_BASE_DIR, "images", "train")
VAL_IMG_DIR = os.path.join(DATASET_BASE_DIR, "images", "val")
TRAIN_LABEL_DIR = os.path.join(DATASET_BASE_DIR, "labels", "train")
VAL_LABEL_DIR = os.path.join(DATASET_BASE_DIR, "labels", "val")

# 3. Train/Validation Split
VAL_SPLIT_RATIO = 0.2

# 4. data.yaml Configuration
# *** IMPORTANT: Define your classes here! ***
# Update NUM_CLASSES to the total number of distinct object types you annotated.
# Update CLASS_NAMES with the names of your classes in the correct order (index 0, 1, 2...).
# Example: If you only annotated 'person' (class 0), use:
# NUM_CLASSES = 1
# CLASS_NAMES = ['person']
# Example: If you annotated 'car' (0) and 'pedestrian' (1), use:
# NUM_CLASSES = 2
# CLASS_NAMES = ['car', 'pedestrian']
NUM_CLASSES = 1 # <<< --- UPDATE THIS
CLASS_NAMES = ['person'] # <<< --- UPDATE THIS LIST (Order matters!)

# Path where the data.yaml file will be created/used
DATA_YAML_PATH = "data.yaml"

# 5. Model and Training Parameters
BASE_MODEL = 'yolov8s-seg.pt' # Adjust if needed (e.g., yolov8s.pt for detection) ot yolo11s-pose.pt
EPOCHS = 50
IMG_SIZE = 640
BATCH_SIZE = 16 # Adjust based on GPU memory
PROJECT_NAME = "custom_yolo_training"
EXPERIMENT_NAME = "run_auto_yaml_v1" # Give it a distinct name

# --- Helper Function ---
def copy_files(file_list, source_img_dir, source_label_dir, dest_img_dir, dest_label_dir):
    """Copies image and corresponding label files."""
    copied_count = 0
    label_missing_count = 0
    print(f"Attempting to copy {len(file_list)} files. Source labels expected in: {source_label_dir}")
    for img_filename in file_list:
        base_name = os.path.splitext(img_filename)[0]
        label_filename = f"{base_name}.txt"
        src_img_path = os.path.join(source_img_dir, img_filename)
        src_label_path = os.path.join(source_label_dir, label_filename)
        dest_img_path = os.path.join(dest_img_dir, img_filename)
        dest_label_path = os.path.join(dest_label_dir, label_filename)

        if os.path.exists(src_img_path):
            shutil.copy2(src_img_path, dest_img_path)
            copied_count += 1
            if os.path.exists(src_label_path):
                shutil.copy2(src_label_path, dest_label_path)
            else:
                label_missing_count += 1
                print(f"Warning: Label file not found for image {img_filename} at {src_label_path}. Image copied without label.")
        else:
            print(f"Warning: Source image not found during copy: {src_img_path}")
            continue
    return copied_count, label_missing_count

# --- 1. Setup Directories ---
print("Creating dataset directories...")
os.makedirs(TRAIN_IMG_DIR, exist_ok=True)
os.makedirs(VAL_IMG_DIR, exist_ok=True)
os.makedirs(TRAIN_LABEL_DIR, exist_ok=True)
os.makedirs(VAL_LABEL_DIR, exist_ok=True)
print(f"Ensured directories exist:\n  {TRAIN_IMG_DIR}\n  {VAL_IMG_DIR}\n  {TRAIN_LABEL_DIR}\n  {VAL_LABEL_DIR}")

# --- 2. Prepare File List and Split ---
print(f"\nScanning source image directory: {SOURCE_IMAGE_DIR}")
if not os.path.isdir(SOURCE_IMAGE_DIR):
    print(f"Error: Source image directory not found at '{SOURCE_IMAGE_DIR}'")
    exit()
if not os.path.isdir(SOURCE_LABEL_DIR):
    print(f"Error: Source label directory not found at '{SOURCE_LABEL_DIR}'")
    exit()

image_extensions = ["*.jpg", "*.jpeg", "*.png", "*.bmp", "*.gif", "*.webp"]
all_image_files = []
for ext in image_extensions:
    all_image_files.extend(glob.glob(os.path.join(SOURCE_IMAGE_DIR, ext)))

all_image_filenames = [os.path.basename(f) for f in all_image_files]
total_images = len(all_image_filenames)

if total_images == 0:
    print(f"Error: No image files found in {SOURCE_IMAGE_DIR} with extensions {image_extensions}")
    exit()

print(f"Found {total_images} images.")
random.shuffle(all_image_filenames)
split_index = int(total_images * (1 - VAL_SPLIT_RATIO))
train_filenames = all_image_filenames[:split_index]
val_filenames = all_image_filenames[split_index:]
print(f"Splitting data: {len(train_filenames)} train ({100*(1-VAL_SPLIT_RATIO):.1f}%), {len(val_filenames)} validation ({100*VAL_SPLIT_RATIO:.1f}%).")

# --- 3. Copy Files to Structured Directories ---
print(f"\nCopying training files (Images from {SOURCE_IMAGE_DIR}, Labels from {SOURCE_LABEL_DIR})...")
train_copied, train_labels_missing = copy_files(train_filenames, SOURCE_IMAGE_DIR, SOURCE_LABEL_DIR, TRAIN_IMG_DIR, TRAIN_LABEL_DIR)
print(f"Copied {train_copied} training images to {TRAIN_IMG_DIR} and labels to {TRAIN_LABEL_DIR}.")
if train_labels_missing > 0:
    print(f"Warning: {train_labels_missing} training label files were missing from {SOURCE_LABEL_DIR}.")

print(f"\nCopying validation files (Images from {SOURCE_IMAGE_DIR}, Labels from {SOURCE_LABEL_DIR})...")
val_copied, val_labels_missing = copy_files(val_filenames, SOURCE_IMAGE_DIR, SOURCE_LABEL_DIR, VAL_IMG_DIR, VAL_LABEL_DIR)
print(f"Copied {val_copied} validation images to {VAL_IMG_DIR} and labels to {VAL_LABEL_DIR}.")
if val_labels_missing > 0:
    print(f"Warning: {val_labels_missing} validation label files were missing from {SOURCE_LABEL_DIR}.")

print("\nData organization complete.")

# --- 4. Create data.yaml File ---
print(f"\nCreating dataset configuration file: {DATA_YAML_PATH}")

# Use absolute paths for robustness in the YAML file
# This ensures YOLO can find the data regardless of where the script is run from
abs_train_img_dir = os.path.abspath(TRAIN_IMG_DIR)
abs_val_img_dir = os.path.abspath(VAL_IMG_DIR)

# Create the dictionary structure for the YAML data
data_yaml_content = {
    'train': abs_train_img_dir,
    'val': abs_val_img_dir,
    'nc': NUM_CLASSES,
    'names': CLASS_NAMES
}

# Write the dictionary to the YAML file
try:
    with open(DATA_YAML_PATH, 'w') as f:
        yaml.dump(data_yaml_content, f, default_flow_style=None, sort_keys=False)
    print(f"Successfully created {DATA_YAML_PATH}")
    print("Content:")
    print(f"  train: {abs_train_img_dir}")
    print(f"  val: {abs_val_img_dir}")
    print(f"  nc: {NUM_CLASSES}")
    print(f"  names: {CLASS_NAMES}")
except Exception as e:
    print(f"Error creating {DATA_YAML_PATH}: {e}")
    exit()

# --- 5. Device Setup ---
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"\nUsing device: {device}")

# --- 6. Load Base Model ---
print(f"Loading base model: {BASE_MODEL}")
if not os.path.exists(BASE_MODEL):
    print(f"Info: Base model '{BASE_MODEL}' not found locally. YOLO will attempt to download it.")
model = YOLO(BASE_MODEL)

# --- 7. Verify data.yaml (Optional check after creation) ---
print(f"\nVerifying content of created {DATA_YAML_PATH}...")
try:
    with open(DATA_YAML_PATH, 'r') as f:
        data_config = yaml.safe_load(f)
        if data_config['nc'] != NUM_CLASSES or data_config['names'] != CLASS_NAMES:
             print(f"Warning: Mismatch between config variables and {DATA_YAML_PATH} content.")
        else:
             print(f"YAML file content seems consistent with script configuration.")
except Exception as e:
    print(f"Error reading back {DATA_YAML_PATH} for verification: {e}")
    # Decide if you want to exit or continue if verification fails
    # exit()

# --- 8. Start Training ---
print("\nStarting model training...")
try:
    results = model.train(
        data=DATA_YAML_PATH,      # Path to the data configuration file
        epochs=EPOCHS,            # Number of training epochs
        imgsz=IMG_SIZE,           # Input image size
        batch=BATCH_SIZE,         # Number of images per batch
        device=device,            # Device to run on ('cuda' or 'cpu')
        project=PROJECT_NAME,     # Directory to save results
        name=EXPERIMENT_NAME,     # Subdirectory name for this specific run
        exist_ok=False            # Prevent overwriting existing runs
    )
    print("Training finished.")
    print(f"Results saved in: {results.save_dir}")

    # --- 9. Post-Training Evaluation (Optional) ---
    print("\nEvaluating model on validation set...")
    best_model_path = os.path.join(results.save_dir, 'weights', 'best.pt')
    if os.path.exists(best_model_path):
        print(f"Loading best model from: {best_model_path}")
        best_model = YOLO(best_model_path)
        metrics = best_model.val()
        print("Validation Metrics obtained.")
        # Inspect metrics object for results (e.g., metrics.box.map50_95)
    else:
        print(f"Warning: Could not find best model weights at {best_model_path}")

except Exception as e:
    print(f"\nAn error occurred during training or evaluation: {e}")
    import traceback
    traceback.print_exc()

print("\nScript finished.")


Creating dataset directories...
Ensured directories exist:
  dataset/images/train
  dataset/images/val
  dataset/labels/train
  dataset/labels/val

Scanning source image directory: extracted_frames
Found 90 images.
Splitting data: 72 train (80.0%), 18 validation (20.0%).

Copying training files (Images from extracted_frames, Labels from output_annotations)...
Attempting to copy 72 files. Source labels expected in: output_annotations
Copied 72 training images to dataset/images/train and labels to dataset/labels/train.

Copying validation files (Images from extracted_frames, Labels from output_annotations)...
Attempting to copy 18 files. Source labels expected in: output_annotations
Copied 18 validation images to dataset/images/val and labels to dataset/labels/val.

Data organization complete.

Creating dataset configuration file: data.yaml
Successfully created data.yaml
Content:
  train: /content/dataset/images/train
  val: /content/dataset/images/val
  nc: 1
  names: ['person']

Using 

100%|██████████| 755k/755k [00:00<00:00, 21.8MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

YOLOv8s-seg summary: 151 layers, 11,790,483 parameters, 11,790,467 gradients, 42.7 GFLOPs

Transferred 411/417 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


100%|██████████| 5.35M/5.35M [00:00<00:00, 95.7MB/s]


...


50 epochs completed in 0.057 hours.
Optimizer stripped from custom_yolo_training/run_auto_yaml_v1/weights/last.pt, 23.8MB
Optimizer stripped from custom_yolo_training/run_auto_yaml_v1/weights/best.pt, 23.8MB

Validating custom_yolo_training/run_auto_yaml_v1/weights/best.pt...
Ultralytics 8.3.126 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8s-seg summary (fused): 85 layers, 11,779,987 parameters, 0 gradients, 42.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.44it/s]


                   all         46        193      0.742      0.964      0.818       0.78      0.742      0.964      0.818        0.7
Speed: 0.1ms preprocess, 4.5ms inference, 0.0ms loss, 2.4ms postprocess per image
Results saved to custom_yolo_training/run_auto_yaml_v1
Training finished.
Results saved in: custom_yolo_training/run_auto_yaml_v1

Evaluating model on validation set...
Loading best model from: custom_yolo_training/run_auto_yaml_v1/weights/best.pt
Ultralytics 8.3.126 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8s-seg summary (fused): 85 layers, 11,779,987 parameters, 0 gradients, 42.4 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2993.8±1160.1 MB/s, size: 385.0 KB)


val: Scanning /content/dataset/labels/val.cache... 34 images, 12 backgrounds, 0 corrupt: 100%|██████████| 46/46 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:02<00:00,  1.10it/s]


                   all         46        193      0.742      0.964      0.818      0.784      0.742      0.964      0.818      0.695
Speed: 4.9ms preprocess, 16.5ms inference, 0.0ms loss, 4.0ms postprocess per image
Results saved to runs/segment/val
Validation Metrics obtained.

Script finished.


In [1]:
import cv2
from ultralytics import YOLO
import os
import torch # Optional: for device check

# --- Configuration ---

# 1. Path to your trained model weights
# This should be the 'best.pt' file located in your training results directory
# e.g., 'custom_yolo_training/run_auto_yaml_v1/weights/best.pt'
# Update this path accordingly!
MODEL_PATH = "/content/custom_yolo_training/run_auto_yaml_v1/weights/best.pt" # <<< --- UPDATE THIS

# 2. Path to the input video file
# Update this to the video you want to process
INPUT_VIDEO_PATH = "/content/race.mp4" # <<< --- UPDATE THIS

# 3. Path where the output video will be saved
# The script will create this file.
OUTPUT_VIDEO_PATH = "output_video.mp4" # <<< --- UPDATE THIS (optional)

# 4. Inference Parameters (Optional)
CONFIDENCE_THRESHOLD = 0.5 # Only show detections with confidence >= 0.5
# Set device: 'cuda' for GPU, 'cpu' for CPU, or None for auto-detect
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu' # Or set manually e.g., 'cpu'

# --- Check if files exist ---
if not os.path.exists(MODEL_PATH):
    print(f"Error: Model weights not found at '{MODEL_PATH}'")
    exit()

if not os.path.exists(INPUT_VIDEO_PATH):
    print(f"Error: Input video not found at '{INPUT_VIDEO_PATH}'")
    exit()

# --- Load Model ---
print(f"Loading model from: {MODEL_PATH}")
try:
    # Load the trained YOLO model
    model = YOLO(MODEL_PATH)
    # Set device
    model.to(DEVICE)
    print(f"Model loaded successfully on device: {DEVICE}")
except Exception as e:
    print(f"Error loading model: {e}")
    exit()

# --- Open Video File ---
print(f"Opening input video: {INPUT_VIDEO_PATH}")
cap = cv2.VideoCapture(INPUT_VIDEO_PATH)

# Check if video opened successfully
if not cap.isOpened():
    print(f"Error: Could not open video file {INPUT_VIDEO_PATH}")
    exit()

# --- Get Video Properties ---
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print(f"Video properties: {frame_width}x{frame_height} @ {fps:.2f} FPS, {total_frames} frames")

# --- Define the codec and create VideoWriter object ---
# Use 'mp4v' for .mp4 output, or 'XVID' for .avi
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(OUTPUT_VIDEO_PATH, fourcc, fps, (frame_width, frame_height))

print(f"Output video will be saved to: {OUTPUT_VIDEO_PATH}")

# --- Process Video Frame by Frame ---
frame_count = 0
while True:
    # Read a frame from the video
    ret, frame = cap.read()

    # If frame is not read correctly (end of video), break the loop
    if not ret:
        print("Reached end of video or failed to read frame.")
        break

    frame_count += 1
    print(f"Processing frame {frame_count}/{total_frames}...")

    # --- Run YOLO Inference ---
    # Perform prediction on the current frame
    # 'stream=True' can be more memory efficient for videos but requires handling differently
    # Here we process frame by frame, so stream=False is fine.
    # 'conf' sets the confidence threshold for detections.
    # 'verbose=False' suppresses detailed console output for each frame.
    results = model.predict(frame, conf=CONFIDENCE_THRESHOLD, device=DEVICE, verbose=False)

    # --- Get Annotated Frame ---
    # results[0].plot() returns the frame with bounding boxes/masks drawn on it
    # Ensure you are using a recent version of ultralytics for this method
    annotated_frame = results[0].plot()

    # --- Write Frame to Output Video ---
    out.write(annotated_frame)

    # --- Display Frame (Optional) ---
    # Uncomment the lines below to display the processed frames in a window
    # cv2.imshow('YOLO Inference', annotated_frame)
    # # Press 'q' to exit the display window early
    # if cv2.waitKey(1) & 0xFF == ord('q'):
    #     print("Exiting display...")
    #     break

# --- Release Resources ---
print("Releasing video resources...")
cap.release() # Release the input video capture object
out.release() # Release the output video writer object
cv2.destroyAllWindows() # Close any OpenCV display windows

print(f"\nProcessing complete. Output video saved to: {OUTPUT_VIDEO_PATH}")


Loading model from: /content/custom_yolo_training/run_auto_yaml_v1/weights/best.pt
Model loaded successfully on device: cuda
Opening input video: /content/race.mp4
Video properties: 1920x1080 @ 30.00 FPS, 538 frames
Output video will be saved to: output_video.mp4
Processing frame 1/538...
Processing frame 2/538...
Processing frame 3/538...
Processing frame 4/538...
Processing frame 5/538...
Processing frame 6/538...
Processing frame 7/538...
Processing frame 8/538...
Processing frame 9/538...
Processing frame 10/538...
Processing frame 11/538...
Processing frame 12/538...
Processing frame 13/538...
Processing frame 14/538...
Processing frame 15/538...
Processing frame 16/538...
Processing frame 17/538...
Processing frame 18/538...
Processing frame 19/538...
Processing frame 20/538...
Processing frame 21/538...
Processing frame 22/538...
Processing frame 23/538...
Processing frame 24/538...
Processing frame 25/538...
Processing frame 26/538...
Processing frame 27/538...
Processing frame

In [ ]:
!zip -r /content/output_annotations.zip /content/output_annotations

  adding: content/output_annotations/ (stored 0%)
  adding: content/output_annotations/frame_00312.txt (deflated 84%)
  adding: content/output_annotations/frame_00384.txt (deflated 85%)
  adding: content/output_annotations/frame_00060.txt (deflated 78%)
  adding: content/output_annotations/frame_00390.txt (deflated 85%)
  adding: content/output_annotations/frame_00096.txt (deflated 78%)
  adding: content/output_annotations/frame_00408.txt (deflated 85%)
  adding: content/output_annotations/frame_00030.txt (deflated 79%)
  adding: content/output_annotations/frame_00510.txt (deflated 83%)
  adding: content/output_annotations/frame_00204.txt (deflated 84%)
  adding: content/output_annotations/frame_00372.txt (deflated 85%)
  adding: content/output_annotations/frame_00306.txt (deflated 85%)
  adding: content/output_annotations/frame_00072.txt (deflated 79%)
  adding: content/output_annotations/frame_00504.txt (deflated 83%)
  adding: content/output_annotations/frame_00210.txt (deflated 84%

In [ ]:
from google.colab import files
files.download("/content/output_annotations.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>